# Geothermobarometry lab



# Setup

In [1]:
try
    using StatGeochem
catch
    println("StatGeochem not installed; installing...")
    using Pkg
    Pkg.add(PackageSpec(url="https://github.com/brenhinkeller/StatGeochem.jl"))
    using StatGeochem
end

using Plots

┌ Info: Recompiling stale cache file /Users/gailin/.julia/compiled/v1.2/StatGeochem/Ht7Cf.ji for StatGeochem [df4de05a-b714-11e8-3c2a-c30fb13e804c]
└ @ Base loading.jl:1240
┌ Warning: Package SpecialFunctions does not have Libdl in its dependencies:
│ - If you have SpecialFunctions checked out for development and have
│   added Libdl as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with SpecialFunctions
└ Loading Libdl into SpecialFunctions from project dependency, future warnings for SpecialFunctions are suppressed.
  likely near /Users/gailin/dartmouth/crustal_structure/StatGeochem.jl/src/StatGeochem.jl:41
  likely near /Users/gailin/dartmouth/crustal_structure/StatGeochem.jl/src/resources/Elevation.jl:3
  likely near /Users/gailin/dartmouth/crustal_structure/StatGeochem.jl/src/resources/Elevation.jl:3
┌ Warning: The call to compilecache failed to create a usable precompiled cache file

In [2]:
## --- Configure

# Absolute paths to perplex resources
perplexdir = joinpath(resourcepath,"perplex-stable")
scratchdir = "./scratch/" # Location of directory to store output files

# Attempt to install perplex, if not already extant
if !isfile(joinpath(perplexdir,"vertex"))
    # Make sure resourcepath exists
    run(`mkdir -p $resourcepath`)

    # Try to compile PerpleX from source; if that fails, try to download linux binaries
    try
        # Check if there is a fortran compiler
        run(`gfortran -v`)

        # Download Perplex v6.8.7 -- known to work with interface used here
        file = download("https://storage.googleapis.com/statgeochem/perplex-6.8.7-source.zip", 
            joinpath(resourcepath,"perplex-stable.zip"))

        # # For a more updated perplex version, you might also try
        # file = download("https://petrol.natur.cuni.cz/~ondro/perplex-sources-stable.zip", 
        #       joinpath(resourcepath,"perplex-stable.zip"))

        run(`unzip -u $file -d $resourcepath`) # Extract
        system("cd $perplexdir; make") # Compile
    catch
        @warn "Failed to compile from source, trying precompiled linux binaries instead"
        run(`mkdir -p $perplexdir`)
        file = download("https://petrol.natur.cuni.cz/~ondro/Perple_X_6.8.7_Linux_64_gfortran.tar.gz",
            "perplex-6.8.7-linux.tar.gz")
        run(`tar -xzf $file -C $perplexdir`)
    end
else 
    println("Perplex already installed")
end

Perplex already installed


# Lab

In [3]:
## Configuration for perple_x 

# Which phases to consider for thermodynamic calculation? 
solution_phases = "O(HP)\nOpx(HP)\nOmph(GHP)\nGt(HP)\noAmph(DP)"*
    "\nGlTrTsPg\nT\nB\nAnth\nChl(HP)\nBio(TCC)\nMica(CF)\nCtd(HP)"*
    "\nIlHm(A)\nSp(HP)\nSapp(HP)\nSt(HP)\nfeldspar\nDo(HP)\nF\n";

excludes = "";

# TODO real granite average 
granite = [74.14,0.25,14.9,2.1146,0.5,4.8,3.15,0.08,1.8386,0.4407];
granite2 = [66.48,0.48,15.38,3.6442,2.33,3.11,3.49,3.89,1.8386,0.4407];

## QUESTION FOR STUDENTS: What changes when we consider some actual granites? 


In [4]:
perplex_configure_pseudosection(perplexdir, scratchdir, granite, xnodes=10, ynodes=10, dataset="hpha02ver.dat")

0

In [ ]:
ans = perplex_query_system(perplexdir, scratchdir)